In [2]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as T

import pickle


In [3]:
# Device configuration

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print('device:', device)

device: cuda


In [4]:
from IPython.display import HTML, display


# Custom IPython progress bar for training
class ProgressMonitor(object):

    tmpl = """
        <table style="width: 100%;">
            <tbody>
                <tr>
                    <td style="width: 30%;">
                     <b>Loss: {loss:0.4f}</b> &nbsp&nbsp&nbsp {value} / {length}
                    </td>
                    <td style="width: 70%;">
                        <progress value='{value}' max='{length}', style='width: 100%'>{value}</progress>
                    </td>
                </tr>
            </tbody>
        </table>
        """

    def __init__(self, length):
        self.length = length
        self.count = 0
        self.display = display(self.html(0, 0), display_id=True)

    def html(self, count, loss):
        return HTML(self.tmpl.format(length=self.length, value=count, loss=loss))

    def update(self, count, loss):
        self.count += count
        self.display.update(self.html(self.count, loss))

In [5]:
transform_train = T.Compose( [T.RandomCrop(32, padding=4), T.ToTensor(), T.Normalize( (0.5, 0.5, 0.5), (0.5, 0.5, 0.5) )] )
transform_test = T.Compose( [T.ToTensor(), T.Normalize( (0.5, 0.5, 0.5), (0.5, 0.5, 0.5) )] )

train_set = torchvision.datasets.CIFAR10('./data', train=True, download=True, transform=transform_train )
test_set = torchvision.datasets.CIFAR10('./data', train=False, download=True, transform=transform_test )

classes = train_set.classes

100%|██████████| 170498071/170498071 [00:19<00:00, 8623639.12it/s] 


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [6]:
print(train_set.data.shape)
print(test_set.data.shape)

(50000, 32, 32, 3)
(10000, 32, 32, 3)


In [9]:
# 과제 1- SimpleCNN의 오류를 없애라!
# Hint- matrix의 size를 주의! maxpooling은 size를 1/2배한다. filter의 size도 중요

class SimpleCNN(nn.Module):

    def __init__(self):
        super().__init__()

        self.conv_layers = nn.Sequential(


            #### 입력 이미지 크기 32x32x3

            nn.Conv2d( in_channels=3, out_channels=64, kernel_size=3, padding=1 ),#out_channel: 필터의 개수
            nn.BatchNorm2d(64),
            nn.ReLU(),

            nn.Conv2d( in_channels=64, out_channels=64, kernel_size=3, padding=1 ),
            nn.BatchNorm2d(64),
            nn.ReLU(),

            nn.MaxPool2d(2),

            nn.Conv2d( in_channels=64, out_channels=128, kernel_size=3, padding=1 ),
            nn.BatchNorm2d(128),
            nn.ReLU(),

            nn.MaxPool2d(2),


        )

        self.fc_layers = nn.Sequential(

            nn.Linear( 128 * 8 * 8, 500), #맨 앞 128이 채널 수, 다음 4가 이미지의 크기
            nn.ReLU(),

            nn.Linear(500, 10), #마지막은 class의 개수가 10개이므로 10으로 맞춰야함

        )


    def forward(self, x):

        x = self.conv_layers(x)

        x = x.view( x.size(0), -1 ) # flatten

        x = self.fc_layers(x)

        return x

In [10]:
# 모델 테스트
# 텐서의 사이즈가 (7, 10)이 나오면 성공
# 현재는 오류가 뜨는 상황! matrix size를 잘 맞춰서 이 코드가 정상적으로 구동되면 성공입니다.

temp = SimpleCNN()
output = torch.randn( 7, 3, 32, 32)

print( temp(output).size() )


torch.Size([7, 10])


여기까지 진행하고! ipynb 파일 제출해주세요.